In [24]:
corenlp_dir = './corenlp'

import os
os.environ["CORENLP_HOME"] = corenlp_dir

In [ ]:
from stanza.server import CoreNLPClient
client = CoreNLPClient(
    annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner', 'openie'], 
    memory='4G', 
    endpoint='http://localhost:9000',
    be_quiet=True)
print(client)

client.start()
import time; time.sleep(10)

In [2]:
import pandas as pd
import json
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm

def load_data(file):
    with open(file) as f:
      data = json.load(f)
    return(data)

def save_data(file, data):
    with open(file, 'w', encoding="utf-8") as f:
        json.dump(data, f, indent = 4)

def repl(data):
    data = data.split(',')
    for i in range(len(data)):
        data[i] = data[i].replace("[", "")
        data[i] = data[i].replace("]", "")
        data[i] = data[i].replace("'", "")
        data[i] = data[i].replace('"', "")
        data[i] = data[i].strip()
    return data


def update_list(data, idd):
    for i in idd:
        data['food'][i] = list(set(data['food'][i]))
        data['count_f'][i] = len(data['food'][i])
        data['med'][i] = list(set(data['med'][i]))
        data['count_m'][i] = len(data['med'][i])
    return data

In [ ]:
df = pd.read_csv("bertowy_model_z_food_med.csv")
good_df = df[(df['count_f'] != 0) & (df['count_m'] != 0)]
numb_id = good_df.index.tolist()

text_list = []

for i in numb_id:
    good_df['med'][i] = repl(good_df['med'][i])
    good_df['food'][i] = repl(good_df['food'][i])
    text_list.append(good_df['text'][i])


update_list(good_df, numb_id)

In [27]:
def get_triples(text):
  fin_rel = []
  result = 0
  relation = []
  triples = []
  document = client.annotate(text, output_format='json') 
  for sentence in document['sentences']:
    for triple in sentence['openie']:
      triples.append({
        'Subject': triple['subject'],
        'Relation': triple['relation'],
        'Object': triple['object']
      })
      relation.append(triple['relation'])
  

  print("Text: ")
  print(text)
  print("Triples: ")
  for i in range(len(triples)):
    print(triples[i])
  
  return triples

In [ ]:
list_entitites_food = []
list_entitites_disease = []
list_id = []
for i in numb_id:
    list_entitites_food.append(good_df['food'][i])
    list_entitites_disease.append(good_df['med'][i])
    list_id.append(good_df['id'][i])


In [ ]:
triples = [get_triples(i) for i in tqdm(text_list)]

In [19]:
def checking_triples(triples, numb):    
    count_triples = 0
    final_list = []
    for i in range(len(triples)):
        count_triples += len(triples[i])
        dummy = 0
        for k in range(len(triples[i])):
            for j in range(len(list_entitites_food[i])):
                for l in range(len(list_entitites_disease[i])):
                    if numb == 1:
                        if ((list_entitites_food[i][j] == triples[i][k]['subject']) and (list_entitites_disease[i][l] == triples[i][k]['object'])):
                            final_list.append({"triples": triples[i][k], "id": list_id[i]})
                            
                    if numb == 2:
                        if ((list_entitites_food[i][j] in triples[i][k]['subject']) and (list_entitites_disease[i][l] in triples[i][k]['object'])):
                            final_list.append({"triples": triples[i][k], "id": list_id[i]})
                            
                    if numb == 3:
                        if ((list_entitites_food[i][j] in triples[i][k]['subject']) and (list_entitites_disease[i][l] in triples[i][k]['object'])):
                            final_list.append({"triples": triples[i][k], "id": list_id[i]})
                            dummy = 1
                            if dummy == 1:
                                break
                        if dummy == 1:
                            break
                    if dummy == 1:
                        break
                if dummy == 1:
                    break
            if dummy == 1:
                break
            
    print("Rozmiar poczatkowy: ", count_triples)
    print("Rozmiar końcowy: ", len(final_list))
    print("Procent poprawnych danych: {:.2f}%".format((100*len(final_list))/count_triples))
    print("Procent zwróconych trójek: {:.2f}%".format((100*len(final_list))/len(triples)))

    return final_list

In [ ]:
final_triples = checking_triples(triples,3)

In [1]:
def create_df(data):

    subject_list = []
    relation_list = []
    object_list = []
    idiki_list = []

    for i in range(len(data)):
        subject_list.append(data[i]['triples']['subject'])
        relation_list.append(data[i]['triples']['relation'])
        object_list.append(data[i]['triples']['object'])
        idiki_list.append(data[i]['id'])

    df = pd.DataFrame({"subject" : subject_list,  "relation" : relation_list, "object" : object_list, "id" : idiki_list})

    return df

In [ ]:
final_df = create_df(final_triples)
final_df.to_csv('openie_final_result.csv', index = False)

In [64]:

def make_graph_relation(data, word):
    G=nx.from_pandas_edgelist(data[data['relation']==word], "subject", "object", 
                            edge_attr=True, create_using=nx.MultiDiGraph())

    plt.figure(figsize=(15,15))
    pos = nx.spring_layout(G, k = 0.5)
    nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
    plt.show()

In [65]:
make_graph_relation(final_df, 'is associated with')